In [1]:
import pandas as pd
import requests
from lxml import etree

# URL of the XML file
url = "https://www.styleisnow.com/feeds/stock_xml_incremental.xml"


def get_data():

    # Send a GET request to fetch the XML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML content
        root = etree.fromstring(response.content)
        
        # Find all <item> elements
        items = root.findall('.//item')
        
        # Convert the <item> elements to a list of dictionaries
        data = []
        for item in items:
            item_data = {}
            for child in item:
                # Remove the namespace from the tag
                tag_name = str(child.tag).split('}')[-1]  # Splits the tag at '}' and takes the part after it
                item_data[tag_name] = child.text  # Store the tag's text content
            data.append(item_data)
        
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(data)[['id', 'size', 'variant', 'qty']]
        df.rename(columns={'id': 'SKU FULL'}, inplace=True)
        return df
    else:
        print("Failed to retrieve the XML content")
        return None
    

In [2]:
df = get_data()
df

,SKU FULL,size,variant,qty
0,232527ASC000002-9760R-os,OS,ROSE BLUSH BLACK,3
1,232417LSN000008-1W000-44,44,OPTICAL WHITE,3
2,232400ABS000035-003NE-os,OS,NERO,3
3,232527FPC000001-1000-os,OS,BLACK,5
4,232527NCX000001-9000-41,41,WHITE,1
...,...,...,...,...
143,242571NSN000008-90282-37,37,BLACK/SILVER/ICE,1
144,242571NSN000017-10366-39,39,WHITE/BEIGE/BEIGE BROWN LEO,5
145,242568XSP000063-BLACK-37,37,BLACK,1
146,242A43NSN000007-99024-9,9,BLACK MAGNET,1


In [14]:
if df is not None:
    bags = pd.read_csv('bags.csv')
    other = pd.read_csv('other_products.csv')

C:\Users\dinar\AppData\Local\Temp\ipykernel_12652\4172104025.py:3: DtypeWarning: Columns (8,25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  other = pd.read_csv('other_products.csv')


In [10]:
bags_merged = bags[bags['SKU FULL'].isin(df['SKU FULL'])]
bags_merged['Qty'] = df[df['SKU FULL'].isin(bags_merged['SKU FULL'])]['qty'].values
bags_merged.to_csv('updated_bags.csv', index=False)

C:\Users\dinar\AppData\Local\Temp\ipykernel_12652\2891837526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bags_merged['Qty'] = df[df['SKU FULL'].isin(bags_merged['SKU FULL'])]['qty'].values


In [16]:
other_merged = other[other['SKU FULL'].isin(df['SKU FULL'])]
other_merged['Qty'] = df[df['SKU FULL'].isin(other_merged['SKU FULL'])]['qty'].values
# other_merged
other_merged.to_csv('updated_other.csv', index=False)

C:\Users\dinar\AppData\Local\Temp\ipykernel_12652\2691776299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_merged['Qty'] = df[df['SKU FULL'].isin(other_merged['SKU FULL'])]['qty'].values


In [17]:
other[other['SKU FULL'].isin(df['SKU FULL'])]['Qty']

1909      6.0
2767      5.0
4713      9.0
5678      5.0
7715      9.0
         ... 
91467    14.0
91547     7.0
92429    24.0
92926     8.0
93475    12.0
Name: Qty, Length: 77, dtype: float64

In [18]:
other_merged['Qty']

1909     3
2767     5
4713     1
5678     1
7715     1
        ..
91467    2
91547    1
92429    5
92926    1
93475    1
Name: Qty, Length: 77, dtype: object